In [1]:
import numpy as np
import matplotlib.pylab as plt

from bullet_utils.env import BulletEnvWithGround
from robot_properties_solo.solo12wrapper import Solo12Robot, Solo12Config

import mim_control_cpp

from dynamic_graph_head import ThreadHead, SimHead, SimVicon, HoldPDController



ModuleNotFoundError: No module named 'mim_control_cpp'